In [1]:
!ls

archive.zip  sample_data


In [2]:
!unzip archive.zip

Archive:  archive.zip
  inflating: IMDB Dataset.csv        


In [3]:
!ls

 archive.zip  'IMDB Dataset.csv'   sample_data


In [21]:
import numpy as np
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [5]:
temp_df = pd.read_csv('IMDB Dataset.csv')

In [6]:
df = temp_df.iloc[:10000]

In [7]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [8]:
df.drop_duplicates(inplace=True)

/tmp/ipython-input-3006716147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [9]:
import re
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text

In [10]:
df['review'] = df['review'].apply(remove_tags)

/tmp/ipython-input-2336150696.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(remove_tags)


In [11]:
df['review'] = df['review'].apply(lambda x:x.lower())

/tmp/ipython-input-740760900.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x:x.lower())


In [14]:
from nltk.corpus import stopwords

sw_list = stopwords.words('english')
df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

/tmp/ipython-input-3434098104.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))


In [15]:
df['review']

,review
0,one reviewers mentioned watching 1 oz episode ...
1,wonderful little production. filming technique...
2,thought wonderful way spend time hot summer we...
3,basically there's family little boy (jake) thi...
4,"petter mattei's ""love time money"" visually stu..."
...,...
9995,"fun, entertaining movie wwii german spy (julie..."
9996,"give break. anyone say ""good hockey movie""? kn..."
9997,movie bad movie. watching endless series bad h...
9998,"movie probably made entertain middle school, e..."


In [17]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 62.7 MB/s eta 0:00:00


In [18]:
import gensim

In [19]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [22]:
story = []
for doc in df['review']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))

In [23]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [24]:
model.build_vocab(story)

In [25]:
model.train(story, total_examples=model.corpus_count, epochs=model.epochs)

(5849293, 6186875)

In [26]:
len(model.wv.index_to_key)

31845

In [27]:
def document_vector(doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [28]:
document_vector(df['review'].values[0])

array([ 0.02045583,  0.33881053,  0.15368609, -0.06286207, -0.01226318,
       -0.5402649 ,  0.1505072 ,  0.78528196, -0.21287699, -0.10335149,
       -0.07791436, -0.5526836 ,  0.07040915,  0.22298321,  0.05356769,
       -0.26142925,  0.09334756, -0.55058575,  0.04483437, -0.68776774,
        0.1570014 ,  0.09500857,  0.20600304, -0.12804924, -0.24882111,
       -0.19372705, -0.27649617, -0.2937346 , -0.34415197,  0.02701614,
        0.37897182,  0.04490772,  0.09005866, -0.27289462, -0.21394889,
        0.2729302 ,  0.06650811, -0.40074587, -0.22884052, -0.7034727 ,
        0.17487524, -0.39995334, -0.24890299,  0.03927675,  0.42116317,
       -0.23268682, -0.3867623 , -0.14765015,  0.17725496,  0.30337355,
        0.13409749, -0.26462024, -0.18700509, -0.16424985, -0.4799713 ,
        0.0421933 ,  0.40843734,  0.10237014, -0.34271705,  0.07219   ,
       -0.02467354,  0.18403232, -0.15115225, -0.0171901 , -0.41710773,
        0.35771298, -0.06293826,  0.22434214, -0.6100963 ,  0.29

In [29]:
from tqdm import tqdm

In [30]:
X = []
for doc in tqdm(df['review'].values):
    X.append(document_vector(doc))

100%|██████████| 9983/9983 [06:24<00:00, 25.99it/s]


In [31]:
X = np.array(X)

In [32]:
X[0]

array([ 0.02045583,  0.33881053,  0.15368609, -0.06286207, -0.01226318,
       -0.5402649 ,  0.1505072 ,  0.78528196, -0.21287699, -0.10335149,
       -0.07791436, -0.5526836 ,  0.07040915,  0.22298321,  0.05356769,
       -0.26142925,  0.09334756, -0.55058575,  0.04483437, -0.68776774,
        0.1570014 ,  0.09500857,  0.20600304, -0.12804924, -0.24882111,
       -0.19372705, -0.27649617, -0.2937346 , -0.34415197,  0.02701614,
        0.37897182,  0.04490772,  0.09005866, -0.27289462, -0.21394889,
        0.2729302 ,  0.06650811, -0.40074587, -0.22884052, -0.7034727 ,
        0.17487524, -0.39995334, -0.24890299,  0.03927675,  0.42116317,
       -0.23268682, -0.3867623 , -0.14765015,  0.17725496,  0.30337355,
        0.13409749, -0.26462024, -0.18700509, -0.16424985, -0.4799713 ,
        0.0421933 ,  0.40843734,  0.10237014, -0.34271705,  0.07219   ,
       -0.02467354,  0.18403232, -0.15115225, -0.0171901 , -0.41710773,
        0.35771298, -0.06293826,  0.22434214, -0.6100963 ,  0.29

In [33]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

y = encoder.fit_transform(df['sentiment'])
y

array([1, 1, 1, ..., 0, 0, 1])

In [34]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [35]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [36]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7776664997496244